![Figura 01 - IBGE](https://upload.wikimedia.org/wikipedia/pt/8/8c/Ibge-logo.png)

O Instituto Brasileiro de Geografia e Estatística (IBGE) é um instituto público da administração federal brasileira criado em 1934 e instalado em 1936 com o nome de Instituto Nacional de Estatística seu fundador e grande incentivador foi o estatístico Mário Augusto Teixeira de Freitas. O nome atual data de 1938. A sede do IBGE está localizada na cidade do Rio de Janeiro.

O IBGE tem atribuições ligadas às geociências e estatísticas sociais, demográficas e econômicas, o que inclui realizar censos e organizar as informações obtidas nesses censos, para suprir órgãos das esferas governamentais federal, estadual e municipal, e para outras instituições e o público em geral.
(https://pt.wikipedia.org/wiki/Instituto_Brasileiro_de_Geografia_e_Estat%C3%ADstica, 2020).

## Objetivo

Este trabalho tem como objetivo realizar estudos de casos utilizando as bases de dados do IBGE para possíveis correlações entre os dados analisados e dados de qualquer outra área de negócio da empresa. Nele encontramos as informações sobre quantidade da população por cada Distrito do Brasil, áreas de ponderação e setores censitários.

### Importação das bibliotecas

In [2]:
import pandas as pd
import requests
import json
import os
import numpy as np
from dbfread import DBF
import re

## Dados de Distritos

Requisição dos dados de distritos a partir da API disponibilizada pelo IBGE no link: https://servicodados.ibge.gov.br/api/docs

**Obs:** Obtém o conjunto de distritos do Brasil a partir dos respectivos identificadores

In [3]:
try:
    req_distritos = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/distritos')
except:
    print('Erro na conexão')
    exit()

Utilizando a biblioteca **json** para fazer a transformação da requisição em json.

In [4]:
dict_dados = json.loads(req_distritos.text)

Os dados disponibilizados pela API estão em um formato de dicionário dentro de outros dicionários, tendo em vista este problema, foi criado uma função para desencadear o objeto e transformar em uma lista de listas.

In [5]:
def dict_for_list(dicionario):
    lista_global = []
    for i in range(0,len(dicionario)):
        lista_local = []
        for j in dicionario[i]:
            if j != 'municipio':
                lista_local.append(dicionario[i][j])
            else:
                for k in dicionario[i][j]:
                    if k != 'microrregiao':
                        lista_local.append(dicionario[i][j][k])
                    else:
                        for l in dicionario[i][j][k]:
                            if l != 'mesorregiao':
                                lista_local.append(dicionario[i][j][k][l])
                            else:
                                for m in dicionario[i][j][k][l]:
                                    if m != 'UF':
                                        lista_local.append(dicionario[i][j][k][l][m])
                                    else:
                                        for n in dicionario[i][j][k][l][m]:
                                            if n != 'regiao':
                                                lista_local.append(dicionario[i][j][k][l][m][n])
                                            else:
                                                for o in dicionario[i][j][k][l][m][n]:
                                                    lista_local.append(dicionario[i][j][k][l][m][n][o])
        lista_global.append(lista_local)
    return lista_global

In [6]:
columns_df_distritos = ['nu_distrito','no_distrito','nu_municipio','no_municipio','nu_microrregiao',
                        'no_microrregiao','nu_mesorregiao','no_mesorregiao','nu_uf','sg_uf','no_uf',
                        'nu_regiao','sg_regiao','no_regiao']

df_distritos = pd.DataFrame(dict_for_list(dict_dados), columns=columns_df_distritos)

## Área de Apuração

Define-se área de ponderação como sendo uma unidade geográfica, formada por um agrupamento de setores censitários, para a aplicação dos procedimentos de calibração das estimativas com as informações
conhecidas para a população como um todo.
*(Censo Demográfico 2000. Agregado por Setores Censitários dos Resultados do
Universo. Documentação do Arquivo. 2ª edição, Rio de Janeiro, 2003, 2011, p. 62).* 

Requisição dos dados sobre **Áreas de apuração** a partir da API SIDRA disponibilizada pelo IBGE no link:  http://api.sidra.ibge.gov.br/
 
 * Documentação para auxiliar na montagem da URL no link: http://api.sidra.ibge.gov.br/home/ajuda
 * Para identificar os números das tabelas ou códigos que representam as categorias foi utilizado o acervo da SIDRA

In [7]:
try:
    req_area_apuracao = requests.get('http://api.sidra.ibge.gov.br/values/t/1497/n18/all/v/allxp/p/all/f/a/h/y?formato=json')
except:
    print('Erro na conexão')
    exit()

Utilizando a biblioteca **json** para fazer a transformação da requisição em json.

In [8]:
dict_area_apuracao = json.loads(req_area_apuracao.text)

Identificação das colunas do objeto json para utilizá-los nos nomes das colunas do DataFrame, posterior a isso, remoção da primeira linha dos dados.

In [9]:
df_area_apuracao_columns = dict_area_apuracao[0]
dict_area_apuracao.pop(0)

{'NC': 'Nível Territorial (Código)',
 'NN': 'Nível Territorial',
 'D1C': 'Área de Ponderação (Código)',
 'D1N': 'Área de Ponderação',
 'D2C': 'Variável (Código)',
 'D2N': 'Variável',
 'D3C': 'Ano (Código)',
 'D3N': 'Ano',
 'D4C': 'Nacionalidade (Código)',
 'D4N': 'Nacionalidade',
 'MC': 'Unidade de Medida (Código)',
 'MN': 'Unidade de Medida',
 'V': 'Valor'}

Criação de duas listas 'values' e 'keys' para realizar a troca de nomes das colunas do DataFrame 'dict_area_apuracao'.

In [10]:
values = []
keys =[]
for i in df_area_apuracao_columns.values():
    values.append(i)
for i in df_area_apuracao_columns:
    keys.append(i)

Troca dos tipos de dados da colunas para inteiro.

In [11]:
df_area_apuracao = pd.DataFrame(dict_area_apuracao)
df_area_apuracao = df_area_apuracao.astype({"NC": int,"D1C": int,"D2C": int,"D3C": int,"D3N": int,"D4C": int,"MC": int,"V": int})

Iteração para realizar a troca de nomes das colunas.

In [ ]:
for i in range(len(keys)):
    df_area_apuracao = area_apuracao.rename(columns={keys[i]: values[i]})

## Setor Censitário

O setor censitário é a unidade territorial estabelecida para fi ns de controle cadastral, formado por área contínua, situada em um único quadro urbano ou rural, com dimensão e número de domicílios que permitam o levantamento por um recenseador. Assim sendo, cada recenseador procederá à coleta de informações tendo como meta a cobertura do setor censitário que lhe é designado.
(https://censo2010.ibge.gov.br/materiais/guia-do-censo/operacao-censitaria.html, 2020).

Requisição dos dados sobre **Setor Censitário** a partir do FTP disponibilizado pelo IBGE no link:  ftp://geoftp.ibge.gov.br

In [12]:
uf = ['ac','al','am','ap','ba','ce','df','es','go','ma','mg','ms','mt','pa',
      'pb','pe','pi','pr','rj','rn','ro','rr','rs','sc','se','sp','to']

for i in uf:
    os.system("wget ftp://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/\
               malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/\
               setores_censitarios_shp/"+i+"/"+i+"_setores_censitarios.zip")

Criação de um arquivo chamado 'uf.txt' com todos os nomes dos arquivos zipados para realizar a descompactação.

In [ ]:
!ls -ltr | grep '_setores_censitarios.zip' | awk '{print $9}' | cut -d\_ -f1 > uf.txt

Iteração para realizar a descompactação dos arquivos zipados criando uma pasta para cada arquivo.

In [ ]:
for i in uf:
    os.system("unzip "+i+"_setores_censitarios.zip -d ./"+i+"_setores_censitarios")

Comando para encontrar todos os arquivos '*.dbf' para criar um arquivo com todos os nomes destes arquivos.

In [ ]:
!find . -name "*.dbf" > dbf.txt

Abrir o arquivo dentro do Python com a lista de todos os nomes de arquivos .dbf.

In [ ]:
dbf = open(r'dbf.txt')

Realizando a leitura de todos os nomes dentro do arquivo 'dbf.txt' e incluindo dentro de uma lista 'setor_censitario'.

In [ ]:
setor_censitario = []
for i in dbf:
    i = re.sub('\n', '',i)
    tabela = DBF(i, load=True, encoding='Windows-1258')
    for j in range(len(tabela.records)):
        setor_censitario.append(tabela.records[j].values())

Transformando a lista 'setor_censitario' em um DataFrame.

In [ ]:
df_setor_censitario = pd.DataFrame(setor_censitario)

## inclusão no banco de dados

Ao terminar a criação dos três DataFrames, distritos, areas de prinderação e setor censitário, os dados poderão ser inseridos em uma base de dados. Neste notebook foi utilizado o banco MS SQL Server na versão 2016. 

Para realizar a conexão com o banco de dados foi utilizado a biblioteca 'pyodbc'.

In [ ]:
import pyodbc

server = 'vm-powerbi' 
database = 'ibge' 
username = 'ibge' 
password = '@q1w2e3r4' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

Comando para ser executado dentro do banco de dados para verificar a existência da tabela de município, caso não exista o código realiza a criação da estrutura no banco de dados.

In [ ]:
cursor.execute("""IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='tb001_municipio')
                      CREATE TABLE dbo.tb001_municipio (
                       nu_distrito int null
                       ,no_distrito varchar(50)
                       ,nu_municipio int null
                       ,no_municipio varchar(50)
                       ,nu_microrregiao int null
                       ,no_microrregiao varchar(50)
                       ,nu_mesorregiao smallint null
                       ,no_mesorregiao varchar(50)
                       ,nu_uf smallint null
                       ,sg_uf char(2)
                       ,no_uf varchar(50)
                       ,nu_regiao smallint null
                       ,sg_regiao char(2)
                       ,no_regiao varchar(50)
                )""")
cnxn.commit()

Criando uma variável 'cols' recebendo todas as colunas do DataFrame 'df_distritos' para a realização do insert na tabela.

In [ ]:
cols = ",".join([str(i) for i in df_distritos.columns.tolist()])

for i,row in df_distritos.iterrows():
    sql = "INSERT INTO dbo.tb001_municipio(" + cols + ") VALUES (" + "?,"*(len(row)-1) + "?)"
    cursor.execute(sql, tuple(row))
cnxn.commit()

Comando para ser executado dentro do banco de dados para verificar a existência da tabela de município, caso não exista o código realiza a criação da estrutura no banco de dados.

In [ ]:
cursor.execute("""IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='tb002_area_apuracao')
                      CREATE TABLE dbo.tb002_area_apuracao (
                        nu_nivel_territorial smallint null,
                        no_nivel_territorial varchar(50) null,
                        nu_area_ponderacao bigint null,
                        no_area_ponderacao varchar(50) null,
                        nu_variavel smallint null,
                        no_variavel varchar(50) null,
                        nu_ano smallint null,
                        no_ano smallint null,
                        nu_nacionalidade int null,
                        no_nacionalidade varchar(50) null,
                        nu_medida smallint null,
                        no_medida varchar(50) null,
                        qt_pessoa int null,
                    )""")
cnxn.commit()

Criando uma variável 'cols' recebendo todas as colunas do DataFrame 'df_area_apuracao' para a realização do insert na tabela.

In [ ]:
cols = ['nu_nivel_territorial','no_nivel_territorial','nu_area_ponderacao','no_area_ponderacao',
        'nu_variavel','no_variavel','nu_ano','no_ano','nu_nacionalidade','no_nacionalidade',
        'nu_medida','no_medida','qt_pessoa']

In [ ]:
for i,row in df_area_apuracao.iterrows():
    sql = "INSERT INTO dbo.tb002_area_apuracao(" + cols + ") VALUES (" + "?,"*(len(row)-1) + "?)"
    #print(sql, tuple(row))
    cursor.execute(sql, tuple(row))
cnxn.commit()

Comando para fechar a conexão com o banco de dados.

In [ ]:
cursor.close()